# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here

# Scale X_train and X_val using StandardScaler

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and validation data
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Convert the scaled arrays back into DataFrame objects
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X.columns)


# Initialize the LinearRegression model
linreg = LinearRegression()

# Fit the model on the scaled training data
linreg.fit(X_train_scaled, y_train)

# Evaluate the model on the training data
train_r2 = linreg.score(X_train_scaled, y_train)
train_mse = mean_squared_error(y_train, linreg.predict(X_train_scaled))

# Evaluate the model on the validation data
val_r2 = linreg.score(X_val_scaled, y_val)
val_mse = mean_squared_error(y_val, linreg.predict(X_val_scaled))

print('Baseline Linear Regression Model:')
print('Train R²:', train_r2)
print('Train MSE:', train_mse)
print('Validation R²:', val_r2)
print('Validation MSE:', val_mse)


Baseline Linear Regression Model:
Train R²: 0.7868344817421309
Train MSE: 1254529861.820535
Validation R²: 0.6375622643038106
Validation MSE: 2398973372.1732197


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [4]:
# Initialize a list to store the results
interaction_results = []

# Generate all possible pairs of columns from X_train
column_pairs = list(combinations(X_train.columns, 2))

for pair in column_pairs:
    # Make copies of X_train and X_val
    X_train_copy = X_train_scaled.copy()
    X_val_copy = X_val_scaled.copy()
    
    # Create the interaction term
    interaction_term_train = X_train_copy[pair[0]] * X_train_copy[pair[1]]
    interaction_term_val = X_val_copy[pair[0]] * X_val_copy[pair[1]]
    
    # Add the interaction term to the dataframes
    interaction_term_name = f"{pair[0]}_x_{pair[1]}"
    X_train_copy[interaction_term_name] = interaction_term_train
    X_val_copy[interaction_term_name] = interaction_term_val
    
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(X_train_copy, y_train)
    
    # Evaluate the model on the validation data
    val_r2 = linreg.score(X_val_copy, y_val)
    
    # Append the results to the data structure
    interaction_results.append((pair, val_r2))

# Sort the results by the R^2 score in descending order
interaction_results.sort(key=lambda x: x[1], reverse=True)

# Get the top 7 interactions
top_7_interactions = interaction_results[:7]

# Print the top 7 interactions
for interaction in top_7_interactions:
    print(f"Interaction: {interaction[0]}, R²: {interaction[1]}")


Interaction: ('LotArea', '1stFlrSF'), R²: 0.7211105666140571
Interaction: ('LotArea', 'TotalBsmtSF'), R²: 0.707164920705011
Interaction: ('LotArea', 'GrLivArea'), R²: 0.6690980823779022
Interaction: ('LotArea', 'Fireplaces'), R²: 0.6529699515652586
Interaction: ('2ndFlrSF', 'TotRmsAbvGrd'), R²: 0.647299489040519
Interaction: ('OverallCond', 'TotalBsmtSF'), R²: 0.642901987923377
Interaction: ('OverallQual', '2ndFlrSF'), R²: 0.6422324294284367


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [5]:
# Loop over top 7 interactions
for interaction in top_7_interactions:
    # Extract column names from data structure
    col1, col2 = interaction[0]
    
    # Construct new column name
    interaction_term_name = f"{col1}_x_{col2}"
    
    # Add new column to X_train and X_val
    X_train_scaled[interaction_term_name] = X_train_scaled[col1] * X_train_scaled[col2]
    X_val_scaled[interaction_term_name] = X_val_scaled[col1] * X_val_scaled[col2]


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [6]:
# Your code here

# Set up data structure

# Initialize a list to store the results
polynomial_results = []

# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in [2, 3, 4]:
        # Make a copy of X_train and X_val
        X_train_copy = X_train_scaled.copy()
        X_val_copy = X_val_scaled.copy()
        
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        poly_train = poly.fit_transform(X_train_copy[[col]])
        poly_val = poly.transform(X_val_copy[[col]])
        
        # Convert the result to a DataFrame
        poly_train_df = pd.DataFrame(poly_train, columns=[f"{col}^d" for d in range(1, degree + 1)])
        poly_val_df = pd.DataFrame(poly_val, columns=[f"{col}^d" for d in range(1, degree + 1)])
        
        # Drop the original column before combining
        X_train_copy = X_train_copy.drop(columns=[col])
        X_val_copy = X_val_copy.drop(columns=[col])
        
        # Add polynomial to data
        X_train_copy = pd.concat([X_train_copy, poly_train_df], axis=1)
        X_val_copy = pd.concat([X_val_copy, poly_val_df], axis=1)
        
        # Fit a linear regression model
        linreg = LinearRegression()
        linreg.fit(X_train_copy, y_train)
        
        # Evaluate the model on the validation data
        val_r2 = linreg.score(X_val_copy, y_val)
        
        # Append to data structure
        polynomial_results.append((col, degree, val_r2))

# Sort the results by the R^2 score in descending order
polynomial_results.sort(key=lambda x: x[2], reverse=True)

# Get the top 7 polynomial terms
top_7_polynomials = polynomial_results[:7]

# Print the top 7 polynomial terms
for poly in top_7_polynomials:
    print(f"Column: {poly[0]}, Degree: {poly[1]}, R²: {poly[2]}")


Column: GrLivArea, Degree: 3, R²: 0.8283344365591123
Column: OverallQual, Degree: 3, R²: 0.8068023565969221
Column: OverallQual, Degree: 4, R²: 0.8033455378866852
Column: OverallQual, Degree: 2, R²: 0.8025833373594087
Column: LotArea, Degree: 4, R²: 0.8003087859249637
Column: 2ndFlrSF, Degree: 3, R²: 0.776751424068592
Column: 2ndFlrSF, Degree: 4, R²: 0.7696907490848108


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [7]:
# Create a dictionary to store the best polynomial term for each column
best_polynomials = {}

# Loop over the top 7 polynomial terms
for poly in top_7_polynomials:
    col, degree, r2 = poly
    # If the column is not already in the dictionary, or if the current polynomial term has a higher R² score
    if col not in best_polynomials or r2 > best_polynomials[col][1]:
        best_polynomials[col] = (degree, r2)

# Loop over the best polynomial terms
for col, (degree, r2) in best_polynomials.items():
    # Make a copy of X_train and X_val
    X_train_copy = X_train_scaled.copy()
    X_val_copy = X_val_scaled.copy()
    
    # Instantiate PolynomialFeatures with relevant degree
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    # Fit polynomial to column and transform column
    poly_train = poly.fit_transform(X_train_copy[[col]])
    poly_val = poly.transform(X_val_copy[[col]])
    
    # Convert the result to a DataFrame
    poly_train_df = pd.DataFrame(poly_train, columns=[f"{col}^{d}" for d in range(1, degree + 1)])
    poly_val_df = pd.DataFrame(poly_val, columns=[f"{col}^{d}" for d in range(1, degree + 1)])
    
    # Drop the original column before combining
    X_train_scaled = X_train_scaled.drop(columns=[col])
    X_val_scaled = X_val_scaled.drop(columns=[col])
    
    # Add polynomial to data
    X_train_scaled = pd.concat([X_train_scaled, poly_train_df], axis=1)
    X_val_scaled = pd.concat([X_val_scaled, poly_val_df], axis=1)


Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [9]:
# Your code here
# Display the first few rows of the updated X_train_scaled to verify the new columns
print(X_train_scaled.head())

# Display the first few rows of the updated X_val_scaled to verify the new columns
print(X_val_scaled.head())

# Display the columns to ensure interaction and polynomial terms are included
print(X_train_scaled.columns)
print(X_val_scaled.columns)


   OverallCond  TotalBsmtSF  1stFlrSF  TotRmsAbvGrd  GarageArea  Fireplaces  \
0    -0.509252    -0.639316 -0.804789      0.250689    0.327629   -0.994820   
1    -0.509252     0.838208  0.641608     -0.365525    0.079146   -0.994820   
2     1.304613    -0.012560 -0.329000     -0.981739   -1.105931   -0.994820   
3     1.304613    -0.339045 -0.609036     -0.981739   -1.134602    0.588023   
4     1.304613    -2.531499 -1.315922      0.250689   -2.281450   -0.994820   

   LotArea_x_1stFlrSF  LotArea_x_TotalBsmtSF  LotArea_x_GrLivArea  \
0            0.092318               0.073336            -0.057254   
1           -0.113385              -0.148128             0.043694   
2            0.081045               0.003094             0.232730   
3            0.230591               0.128368             0.433899   
4            0.014341               0.027589             0.005250   

   LotArea_x_Fireplaces  ...  OverallQual^1  OverallQual^2  OverallQual^3  \
0              0.114116  ...     

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [10]:
# Your code here
# Fit a linear regression model on the updated training data
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

# Calculate the R² score on the training data
train_r2 = linreg.score(X_train_scaled, y_train)

# Calculate the R² score on the validation data
val_r2 = linreg.score(X_val_scaled, y_val)

# Print the R² scores
print(f"Train R²: {train_r2}")
print(f"Validation R²: {val_r2}")


Train R²: 0.8526309398926483
Validation R²: 0.7516461451222044


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [11]:
# Your code here
from sklearn.feature_selection import RFE

# Define a range of n_features_to_select values
n_features_range = [5, 10, 15, 20, 25, 30]

# Loop over the range of n_features_to_select values
for n_features in n_features_range:
    # Initialize RFE with a linear regression estimator
    rfe = RFE(estimator=LinearRegression(), n_features_to_select=n_features)
    
    # Fit RFE on the training data
    rfe.fit(X_train_scaled, y_train)
    
    # Transform the training and validation datasets
    X_train_rfe = rfe.transform(X_train_scaled)
    X_val_rfe = rfe.transform(X_val_scaled)
    
    # Fit a linear regression model on the transformed training data
    linreg = LinearRegression()
    linreg.fit(X_train_rfe, y_train)
    
    # Calculate the R² score on the training data
    train_r2 = linreg.score(X_train_rfe, y_train)
    
    # Calculate the R² score on the validation data
    val_r2 = linreg.score(X_val_rfe, y_val)
    
    # Print the results
    print(f"n_features_to_select: {n_features}")
    print(f"Train R²: {train_r2}")
    print(f"Validation R²: {val_r2}")
    print(f"Number of features selected: {n_features}")
    print("-" * 30)



n_features_to_select: 5
Train R²: 0.776039994126505
Validation R²: 0.635298172527236
Number of features selected: 5
------------------------------
n_features_to_select: 10
Train R²: 0.8235682746440406
Validation R²: 0.7699939539204792
Number of features selected: 10
------------------------------
n_features_to_select: 15
Train R²: 0.8358457626644923
Validation R²: 0.8369035748713005
Number of features selected: 15
------------------------------
n_features_to_select: 20
Train R²: 0.8494610550826857
Validation R²: 0.7996898379311587
Number of features selected: 20
------------------------------
n_features_to_select: 25
Train R²: 0.852630931786299
Validation R²: 0.7517809760135477
Number of features selected: 25
------------------------------
n_features_to_select: 30
Train R²: 0.8526309398926483
Validation R²: 0.7516461451222044
Number of features selected: 30
------------------------------


## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [12]:
# Your code here

# Step 1: Scale X_test using the same scaler fitted on X_train
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Step 2: Add interaction terms to X_test
# Assuming interaction terms were added in a similar manner as in previous steps
# Example: Adding interaction terms for pairs of columns
interaction_cols = [('LotArea', 'OverallQual'), ('TotalBsmtSF', 'GrLivArea')]  # Example pairs

for col1, col2 in interaction_cols:
    X_test_scaled[f'{col1}*{col2}'] = X_test_scaled[col1] * X_test_scaled[col2]

# Step 3: Add polynomial terms to X_test based on the best polynomial terms identified
for col, (degree, r2) in best_polynomials.items():
    # Instantiate PolynomialFeatures with relevant degree
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    # Fit polynomial to column and transform column
    poly_test = poly.fit_transform(X_test_scaled[[col]])
    
    # Convert the result to a DataFrame
    poly_test_df = pd.DataFrame(poly_test, columns=[f"{col}^{d}" for d in range(1, degree + 1)])
    
    # Drop the original column before combining
    X_test_scaled = X_test_scaled.drop(columns=[col])
    
    # Add polynomial to data
    X_test_scaled = pd.concat([X_test_scaled, poly_test_df], axis=1)

# Display the first few rows of the updated X_test_scaled to verify the new columns
print(X_test_scaled.head())


   OverallCond  TotalBsmtSF  1stFlrSF  TotRmsAbvGrd  GarageArea  Fireplaces  \
0     2.211546    -0.007794 -0.299094     -0.365525   -1.019917   -0.994820   
1    -0.509252     0.954980  0.875424      1.483117    1.120866    2.170867   
2     0.397681    -0.129332 -0.407845     -0.981739   -0.561178    0.588023   
3     1.304613    -0.138864 -0.473096      0.250689   -0.274466    2.170867   
4    -0.509252     1.329127  1.201679     -0.365525    2.076573    0.588023   

   LotArea*OverallQual  TotalBsmtSF*GrLivArea  GrLivArea^1  GrLivArea^2  ...  \
0             0.020576               0.007196    -0.923274     0.852435  ...   
1             0.147591               2.017851     2.112978     4.464675  ...   
2             0.134258               0.129516    -1.001427     1.002856  ...   
3             0.048461              -0.033495     0.241209     0.058182  ...   
4             0.336860               0.206334     0.155240     0.024100  ...   

   OverallQual^1  OverallQual^2  OverallQual

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [19]:
# Assuming interaction terms were added in a similar manner as in previous steps
interaction_cols = [('LotArea', 'OverallQual'), ('TotalBsmtSF', 'GrLivArea')]  # Example pairs

# Add interaction terms to X_test_scaled
for col1, col2 in interaction_cols:
    X_test_scaled[f'{col1}*{col2}'] = X_test_scaled[col1] * X_test_scaled[col2]

# Add polynomial terms to X_test based on the best polynomial terms identified
for col, (degree, r2) in best_polynomials.items():
    # Instantiate PolynomialFeatures with relevant degree
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    # Fit polynomial to column and transform column
    poly_test = poly.fit_transform(X_test_scaled[[col]])
    
    # Convert the result to a DataFrame
    poly_test_df = pd.DataFrame(poly_test, columns=[f"{col}^{d}" for d in range(1, degree + 1)])
    
    # Drop the original column before combining
    X_test_scaled = X_test_scaled.drop(columns=[col])
    
    # Add polynomial to data
    X_test_scaled = pd.concat([X_test_scaled, poly_test_df], axis=1)

# Ensure the same columns are present in X_test_scaled as in X_train_val
X_test_scaled = X_test_scaled[X_train_val.columns]


KeyError: 'LotArea'

In [18]:
# Your code here
from sklearn.metrics import mean_squared_error

# Define the number of features to select
n_features_to_select = 20  # Example value, adjust as needed

# Initialize RFE with a linear regression estimator
rfe = RFE(estimator=LinearRegression(), n_features_to_select=n_features_to_select)

# Fit RFE on the combined training and validation data
rfe.fit(X_train_val, y_train_val)

# Transform the combined training and validation datasets
X_train_val_rfe = rfe.transform(X_train_val)
X_test_rfe = rfe.transform(X_test_scaled)

# Fit a linear regression model on the transformed combined training and validation data
linreg = LinearRegression()
linreg.fit(X_train_val_rfe, y_train_val)

# Calculate the R² score on the test data
test_r2 = linreg.score(X_test_rfe, y_test)

# Calculate the MSE on the test data
test_mse = mean_squared_error(y_test, linreg.predict(X_test_rfe))

# Print the R² and MSE values
print(f"Test R²: {test_r2}")
print(f"Test MSE: {test_mse}")


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- LotArea*OverallQual
- TotalBsmtSF*GrLivArea
Feature names seen at fit time, yet now missing:
- 2ndFlrSF_x_TotRmsAbvGrd
- LotArea_x_1stFlrSF
- LotArea_x_Fireplaces
- LotArea_x_GrLivArea
- LotArea_x_TotalBsmtSF
- ...


In [16]:
from sklearn.linear_model import Lasso

# Initialize Lasso with a chosen alpha value
lasso = Lasso(alpha=0.01)  # Example value, adjust as needed

# Fit Lasso on the combined training and validation data
lasso.fit(X_train_val, y_train_val)

# Transform the test dataset using the fitted Lasso model
X_test_lasso = X_test_scaled.loc[:, lasso.coef_ != 0]

# Calculate the R² score on the test data
test_r2 = lasso.score(X_test_scaled, y_test)

# Calculate the MSE on the test data
test_mse = mean_squared_error(y_test, lasso.predict(X_test_scaled))

# Print the R² and MSE values
print(f"Test R²: {test_r2}")
print(f"Test MSE: {test_mse}")


IndexError: Boolean index has wrong length: 26 instead of 21

## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.